# Acknowledgments

The core logic and all Python code in this notebook were written by me. I used an AI assistant to help refine the in-code comments and documentation.


In [69]:
"""
Comprehensive Security Log Management and Access Control Script:

This script simulates a real-world cybersecurity task by performing a
three-phase analysis on log data. It is designed to demonstrate
core Python skills for security applications, including File I/O,
Regex parsing, list filtering, and function-based authentication logic.
"""

# ----------------- DATA SETUP (Simulated Environment) -----------------
# In a real environment, these files would already exist on a server.
# I create them here to make the script self-contained and testable.

'\nComprehensive Security Log Management and Access Control Script:\n\nThis script simulates a real-world cybersecurity task by performing a\nthree-phase analysis on log data. It is designed to demonstrate\ncore Python skills for security applications, including File I/O,\nRegex parsing, list filtering, and function-based authentication logic.\n'

In [70]:
# Cell 1: The RAW Input Log (simulates mixed data from a firewall or server)
raw_log_content = """
INFO: name1 logged in from 192.168.10.20 on device_id_a.
ERROR: Failed attempt by name5 from 10.0.0.5 on device_id_e.
AUDIT: name3 connection from 192.168.10.30 (Internal).
INFO: name2 connection established from 192.168.10.25 (External).
WARN: Suspicious IP 10.0.0.5 trying to access restricted data.
AUDIT: name4 logged in from 192.168.10.35 on device_id_d.
WARN: Failed login attempt from 256.100.100.100 (Suspicious)
ERROR: Bad IP 192.300.1.1 detected from unproxied connection.
INFO: Traffic from 10.0.0.999 seen at firewall.
"""

with open("raw_access_log.txt", "w") as file:
    file.write(raw_log_content)
print("1. Created simulated raw_access_log.txt")

1. Created simulated raw_access_log.txt


In [71]:
# Cell 2: The Master IP Allow List (simulates a security policy file)
master_ips = """192.168.10.20
192.168.10.30
192.168.10.35"""

with open ("master_ip_allow_list.txt", "w") as file:
    file.write (master_ips)
print("2. Created simulated master_ip_allow_list.txt")

2. Created simulated master_ip_allow_list.txt


In [72]:
# Cell 3: Define In-Memory Access Control Lists (Parallel Lists)
# These global lists map approved users to their unique, assigned device ID.
approved_users = ["name1", "name2", "name3", "name4", "name5"]
approved_devices = ["device_id_a", "device_id_b", "device_id_c", "device_id_d", "device_id_e"]

print("3. Defined in-memory user and device lists.")

3. Defined in-memory user and device lists.


In [73]:
# ----------------- PHASE 1: LOG PARSING (Regex Triage) -----------------

import re

# Read the entire content of the raw log file
with open ("raw_access_log.txt", "r") as file:
    log_text = file.read()

# Define the Regex pattern to find all valid IP addresses
ip_pattern = r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}"

# Use re.findall() to extract all IPs and store them in a list
extracted_ips = re.findall(ip_pattern, log_text)

print("\n--- PHASE 1: Extracted IPs from Raw Log ---")
print(extracted_ips)


--- PHASE 1: Extracted IPs from Raw Log ---
['192.168.10.20', '10.0.0.5', '192.168.10.30', '192.168.10.25', '10.0.0.5', '192.168.10.35', '256.100.100.100', '192.300.1.1', '10.0.0.999']


In [74]:
# ----------------- PHASE 1.5: IP VALIDATION -----------------
#
# This phase validates all IPs extracted from the raw log in Phase 1.
# It filters the list to ensure that only well-formed, valid IPv4
# addresses (e.g., 0.0.0.0 to 255.255.255.255) are passed to the next phase.
# Malformed IPs (like 256.1.1.1 or 10.0.0.999) are filtered out.
#

def is_valid_ip(ip_str):
  """
  Validates if a given string is a well-formed IPv4 address.

  Checks for two main conditions:
  1. The address must consist of exactly four octets.
  2. Each octet must be a digit between 0 and 255.

  Args:
      ip_str (str): The IP address string to validate.

  Returns:
      bool: True if the IP is valid, False otherwise.
  """

  # Split the string into potential octets
  octets = ip_str.split('.')

  # 1. Check for exactly four octets
  if len(octets) != 4:
    return False

  # 2. Check each octet's value
  for octet in octets:
    # Octet must be composed of digits
    if not octet.isdigit():
      return False

    # Convert to int and check if in the valid 0-255 range
    octet_val = int(octet)
    if not 0 <= octet_val <= 255:
      return False

  # If all checks pass, the IP is valid
  return True

# --- Run the Validation ---
# Use the function to create the new filtered list
valid_ips = [ip for ip in extracted_ips if is_valid_ip(ip)]

# Print the results to show the function at work
print("\n--- PHASE 1.5: IP Validation Results ---")
print(f"Extracted IPs (Raw): {extracted_ips}")
print(f"Valid IPs (Filtered): {valid_ips}")


--- PHASE 1.5: IP Validation Results ---
Extracted IPs (Raw): ['192.168.10.20', '10.0.0.5', '192.168.10.30', '192.168.10.25', '10.0.0.5', '192.168.10.35', '256.100.100.100', '192.300.1.1', '10.0.0.999']
Valid IPs (Filtered): ['192.168.10.20', '10.0.0.5', '192.168.10.30', '192.168.10.25', '10.0.0.5', '192.168.10.35']


In [75]:
# ----------------- PHASE 2: POLICY FILTERING (Allow List Check) -----------------

# Read the master allow list file and convert its content into a Python list
with open ("master_ip_allow_list.txt", "r") as file:
    allowed_list_text = file.read()

allowed_list = allowed_list_text.split()

# Initialize the suspicious list and implement the filtering loop
suspicious_ips = []
for IP in valid_ips:
    # Check if the extracted IP is NOT in the official allow list
    if IP not in allowed_list:
        suspicious_ips.append(IP)

print("\n--- PHASE 2: Policy Filtering Results ---")
print(f"Suspicious IPs found: {suspicious_ips}")
print("--- End of Filtering ---")


--- PHASE 2: Policy Filtering Results ---
Suspicious IPs found: ['10.0.0.5', '192.168.10.25', '10.0.0.5']
--- End of Filtering ---


In [76]:
# ----------------- PHASE 3: AUTHENTICATION LOGIC (Function) -----------------

def check_login(username, device_id):
    """
    Checks if a user is approved and verifies their assigned device.

    This function uses parallel lists (approved_users and approved_devices)
    to enforce a "User + Device" security policy.

    Args:
        username (str): The username attempting to log in.
        device_id (str): The device ID being used for the login.

    Returns:
        str: A security verdict message (ACCESS GRANTED, DEVICE MISMATCH, ACCESS DENIED).
    """
    if username in approved_users:
        # 1. Find the user's index in the approved_users list
        user_index = approved_users.index(username)

        # 2. Compare the incoming device_id with the assigned device at that index
        if device_id == approved_devices[user_index]:
            return "ACCESS GRANTED: User and device verified."
        else:
            return f"DEVICE MISMATCH: User {username} approved, but device {device_id} is unauthorized. SECURITY ALERT."
    else:
        # 3. Handle case where the username is not in the approved list at all
        return f"ACCESS DENIED: User {username} is not an approved system user. SECURITY ALERT."


In [77]:
# ----------------- PHASE 4: SCRIPT TESTING (Final Output) -----------------

print("\n--- PHASE 4: Authentication Logic Test ---")

# Test 1: SUCCESS - Approved user and correct device
print(check_login("name3", "device_id_c"))

# Test 2: MISMATCH - Approved user, WRONG device
print(check_login("name2", "device_id_d"))

# Test 3: DENIED - User not in the approved list at all
print(check_login("name999", "device_id_x"))


--- PHASE 4: Authentication Logic Test ---
ACCESS GRANTED: User and device verified.
DEVICE MISMATCH: User name2 approved, but device device_id_d is unauthorized. SECURITY ALERT.
ACCESS DENIED: User name999 is not an approved system user. SECURITY ALERT.
